<a href="https://colab.research.google.com/github/dieko95/blackouts-C4V/blob/omar-first-iter/twitter_pretagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated Tagging

## Libraries



In [0]:
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd

## Accessing Data

In [0]:
## Installing package to access
!pip install --upgrade -q google-auth
!pip install --upgrade -q gspread

# For better visualization of text in Pandas DF
pd.set_option('display.max_colwidth', None)

In [0]:
# Authentication to access the untagged spreadhseet
auth.authenticate_user()

# Authenticating to open google sheets
gc = gspread.authorize(GoogleCredentials.get_application_default())

# Open File's sheet 1
worksheet = gc.open('Training Set - #SinLuz Country Classifier').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

In [0]:
# Convert to a DataFrame and render.

tagOriginalDf = pd.DataFrame(rows)

# First row has the column names
  # Rename columns
tagOriginalDf.columns = tagOriginalDf.iloc[0,:]

# Drop Columns
tagOriginalDf = tagOriginalDf.drop(index = 0)

In [6]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Corresponding Section
tags_df = tagOriginalDf.iloc[8500:11000,:].copy()

# Tagged Tweets
pre_tag_df = tags_df[['full_text','concat_text_user_description', 'label_country', 'label_state', 'label_type']].iloc[:271,:].copy()

# Tweets to tag
to_tag_df = tags_df[['full_text','concat_text_user_description', 'label_country', 'label_state', 'label_type']].iloc[271:,:].copy()

## Cleaning Text

In [0]:
def cleaner(df,text_col):
  # to lower

  df[text_col] = df[text_col].str.lower()

  # Convert common spanish accents

  df[text_col] = df[text_col].str.replace("ú", "u")
  df[text_col] = df[text_col].str.replace("ù", "u")
  df[text_col] = df[text_col].str.replace("ü", "u")
  df[text_col] = df[text_col].str.replace("ó", "o")
  df[text_col] = df[text_col].str.replace("ò", "o")
  df[text_col] = df[text_col].str.replace("í", "i")
  df[text_col] = df[text_col].str.replace("ì", "i")
  df[text_col] = df[text_col].str.replace("é", "e")
  df[text_col] = df[text_col].str.replace("è", "e")
  df[text_col] = df[text_col].str.replace("á", "a")
  df[text_col] = df[text_col].str.replace("à", "a")
  df[text_col] = df[text_col].str.replace("ñ", "gn")

  return df


to_tag_df = cleaner(to_tag_df, 'concat_text_user_description')
to_tag_df = cleaner(to_tag_df, 'full_text')

## Functions 

- Classify label_type (service reported)
  - Extracting pound signs (\#)
- Classify Country
  - Matches any state? 
  - has keyword 'edo' or 'estado' in it?
  - Follows any of the common accounts?
- Classify State
  - Match with list of venezuela states
  - We can use a list of venezuelan cities as well 

### Classifying Label Type

#### Hashtags

* \#SinLuz






In [0]:
import re

# sinluz         670
# ahora           68
# sinagua         66
# apagon          65
# singasolina     53

hashtags = pd.Series(re.findall('#(\w+)', to_tag_df.concat_text_user_description.to_string())).copy()

# hashtags.value_counts()[hashtags.value_counts() > 10]

# hashtags[hashtags == 'sinl']



### Tagging Country

  - Matches any state? 
  - has keyword 'edo' or 'estado' in it?
  - Follows any of the common accounts?

*Notes*
  - For this section I will use the tweet's original text. If I include the user description it can add noise because a user can be reporting about a power outage of another state (e.g., I'm from caracas and reporting a power outage in Zulia)
  

In [0]:
# # Read csv with Venezuela's administrative distribution
# geo_df = pd.read_csv('/content/drive/My Drive/monitor_ciudad/cod.csv')

# # Clean Columns
# cols = ['parroquia', 'nombrepob', 'estado','nom_mun']

# for col in cols:
#   geo_df = cleaner(geo_df, col)

# # # Unique pob name
# #   # CHECK ENCODING ERROR WITH JESUS  car�pano
# # # geo_df.nombrepob.unique()

# state = geo_df.estado.unique().tolist()

# muni = geo_df.nom_mun.unique().tolist()


# indices = to_tag_df.concat_text_user_description.str.contains('|'.join(state), case = False) # 6754

# indices = to_tag_df.concat_text_user_description.str.contains('\sedo', case = False)

# ## View results
# to_tag_df[indices]

# ## Tag Venezuela 
# to_tag_df.loc[indices,'label_country'] = 'venezuela'

# _multiple = []
# for index,row in to_tag_df.loc[to_tag_df.label_country == 'venezuela', 'full_text'].iteritems():
#   _lst = []

#   for state_name in state:
#     if state_name in row:
#       _lst.append(1)
#       if len(_lst) > 1:
#         _multiple.append(index)
#       else:
#         next
#       # print(len(_lst), '------', index)
#     else:
#       next

# print(_multiple)


# Quality Control 



# tst = 'Todavía #SinLuz y son las 6:30  pm nueve horas sin luz en San Cristóbal  estado tachira sector los #kioskos #SinLuz vía seguro social exigimos el servicio ya dejen la burla y el cinismo al pueblo hijos de putas @CORPOELECinfo @corpoelectachir @NicolasMaduro @dcabellor "Luis Suarez" "LuisSua45172692" "soy abogado egresado de la universidad Javeriana de Bogotá Colombia  especialista en derecho internacional"'
# tst = cleaner(pd.DataFrame({'col':[tst]}), 'col').to_string()
# # If there's more than one element in state then it's multiple
# for i in state:
  # if i in tst:
  #   print(i)
  # else:
  #   next




#### Municipality

In [20]:

# There's a municipality called
  # Democracy 

# indices = to_tag_df.concat_text_user_description.str.contains('|'.join(muni), case = False) # 6754

# to_tag_df[indices]

NameError: ignored

In [14]:
# 486 tweets with hashtags Out of 2228 tweets



indices = to_tag_df.concat_text_user_description.str.contains('venezuela', case = False) # 6754

to_tag_df[indices]

,full_text,concat_text_user_description,label_country,label_state,label_type
8775,en nombre de mi comandante eterno e infinito hugo chavez y de mi amado lider @nicolasmaduro te perdono...\n\n#09sep #venezuelaindetenible #sinluz \n\n@carnetdlapatria\n\n @patria_ve\n\n @mippcivzla\n @nicolasmaduro\n @dcabellor https://t.co/pjopewyftf,"en nombre de mi comandante eterno e infinito hugo chavez y de mi amado lider @nicolasmaduro te perdono...\n\n#09sep #venezuelaindetenible #sinluz \n\n@carnetdlapatria\n\n @patria_ve\n\n @mippcivzla\n @nicolasmaduro\n @dcabellor https://t.co/pjopewyftf ""el camarada salah 5000k"" ""vatolxs"" ""hijo de una tierra de patriotas y de libertadores...radicalmente venezolano/revolucionario/chavista/patriota/soydelsur \n\nciudadano constitucional\n\n#tuiterosvzla""",0,0,
8781,"no pierde vigencia ese chiste ya viejo sobre la situacion en #venezuela \n""¿quien estara jugando #jumanji ?"" \n#hiperinflacion \n#crisispolitica \n#sinluz \n#sinagua \n#calor \ny de paso ahora \n#temblor #sismo \n#9sept","no pierde vigencia ese chiste ya viejo sobre la situacion en #venezuela \n""¿quien estara jugando #jumanji ?"" \n#hiperinflacion \n#crisispolitica \n#sinluz \n#sinagua \n#calor \ny de paso ahora \n#temblor #sismo \n#9sept ""cesar gutierr(e)z (dj)"" ""dj_cesar"" ""venezolano en ccs pongo musica por hobbie, me gusta la musica de los 80 y 90. regueton solo para trabajar. el resto del tiempo trabajo en calidad de sistema(qa)""",0,0,
8787,entre apagones ha estado este dia en #tachira ¿y ustedes tienen electricidad? desde donde nos reportan. \n\n#apagon #venezuela #sinluz #14dic #diciembre #navidad https://t.co/kbtoc2slgz,"entre apagones ha estado este dia en #tachira ¿y ustedes tienen electricidad? desde donde nos reportan. \n\n#apagon #venezuela #sinluz #14dic #diciembre #navidad https://t.co/kbtoc2slgz ""tachira post"" ""tachirapost"" ""medio digital en tachira\n\ninformacion veraz regional, nacional e internacional\n\nenvianos tu reporte o denuncia al dm""",0,0,
8789,"@federicamog @ep_president @unioneuropea #sinagua #sinluz #sosvenezuela \n\ntampoco hay comida a precios accesibles. todo muy caro y el salario y pensiones son $2 al mes. mucha hambre, pobreza, desempleo. operacion exterminio #sosvenezuela","@federicamog @ep_president @unioneuropea #sinagua #sinluz #sosvenezuela \n\ntampoco hay comida a precios accesibles. todo muy caro y el salario y pensiones son $2 al mes. mucha hambre, pobreza, desempleo. operacion exterminio #sosvenezuela ""2c@#duinconsueloc"" ""duinconsueloc"" ""persona de la tercera edad\r\ncreyente en dios y la santisima virgen\r\npreocupada por el destino de mi pais\r\ncuenta de ayuda para todos""",0,0,
8790,@omachitozam @marthamendozap @benhur8a @bernardo_98364 @oraffor @figueroa35 @acastellanosb buen descanso omar y panitas #lluvia #sinluz #sinsegnal,"@omachitozam @marthamendozap @benhur8a @bernardo_98364 @oraffor @figueroa35 @acastellanosb buen descanso omar y panitas #lluvia #sinluz #sinsegnal ""nany 🐦"" ""nanitap_23"" ""lic adm. mencion gerencia industrial •unellez..los jovenes productores ganaderos queremos un cambio en venezuela #tercosdelcampovzla 🐮🐂🐄🐏""",0,0,
...,...,...,...,...,...
10963,"la calima que cubre el valle del turbio en el estado lara presagia elevadas temperaturas que afectaran el consumo electrico en nuestra zona donde no se resolvio nada en plantas termolectricas que dejaron dagnar por negligencia, impericia tecnica y corrupcion @corpoeleclara #sinluz","la calima que cubre el valle del turbio en el estado lara presagia elevadas temperaturas que afectaran el consumo electrico en nuestra zona donde no se resolvio nada en plantas termolectricas que dejaron dagnar por negligencia, impericia tecnica y corrupcion @corpoeleclara #sinluz ""ing.hector rodriguez"" ""hecosw"" ""ing.electricista. estado lara, venezuela produccion de fusibles y conectores electricos, tecnologia venezolana de solucion""",0,0,
10965,"@corpoelecinfo #sinluz desde anoche para varia villanueva del hatiilo

In [0]:
ciudades =  ("""puerto ayacucho|
               pto. ayacucho|
               barcelona|
               san fernando de apure|
               maracay|
               barinas|
               ciudad bolivar|
               valencia|
               san carlos|
               tucupita|
               caracas|
               coro|
               san juan de los morros| 
               barquisimeto|
               merida|
               los teques|
               maturin|
               la asuncion|
               margarita|
               guanare|
               cumana|
               san cristobal|
               trujillo|
               la guaira|
               san felipe|
               maracaibo|
               ciudad ojeda|""" ) #2229 tweets

# ('caracas| maracaibo') #327



In [48]:
indices_w_cities = to_tag_df.concat_text_user_description.str.contains(ciudades, case = False)

#2229 rows with the cities above
to_tag_vzla = to_tag_df[indices_w_cities]

#Tagging the 'label_country' column
to_tag_vzla['label_country'] = 0

,full_text,concat_text_user_description,label_country,label_state,label_type
8772,#sinluz en la bombilla petare nuevamente,"#sinluz en la bombilla petare nuevamente ""..."" ""jrsg_2"" """"",0,0,
8773,delincuentes profesionales #estafa #megaestafa #ratas #cafres #apagon https://t.co/yv4iyincuv,"delincuentes profesionales #estafa #megaestafa #ratas #cafres #apagon https://t.co/yv4iyincuv ""florentino ariza potrozo"" ""soyflorentinoar"" ""que usted no sea una victima mas del delincuente, gonorrea, ladron, rata y estafador decatriple hp alejandro sanclemente mendoza cedula 80424614.""",0,0,
8774,"@omairalabradorm alegria de tisico mi estimada licenciada, otra vez sin energia electrica en #tariba, vamos a ver cuantas horas, amanecera y veremos #sinluz","@omairalabradorm alegria de tisico mi estimada licenciada, otra vez sin energia electrica en #tariba, vamos a ver cuantas horas, amanecera y veremos #sinluz ""ja"" ""jargenisruiz"" ""cada dia que pasa debe ser mejor y mejor!!""",0,0,
8775,en nombre de mi comandante eterno e infinito hugo chavez y de mi amado lider @nicolasmaduro te perdono...\n\n#09sep #venezuelaindetenible #sinluz \n\n@carnetdlapatria\n\n @patria_ve\n\n @mippcivzla\n @nicolasmaduro\n @dcabellor https://t.co/pjopewyftf,"en nombre de mi comandante eterno e infinito hugo chavez y de mi amado lider @nicolasmaduro te perdono...\n\n#09sep #venezuelaindetenible #sinluz \n\n@carnetdlapatria\n\n @patria_ve\n\n @mippcivzla\n @nicolasmaduro\n @dcabellor https://t.co/pjopewyftf ""el camarada salah 5000k"" ""vatolxs"" ""hijo de una tierra de patriotas y de libertadores...radicalmente venezolano/revolucionario/chavista/patriota/soydelsur \n\nciudadano constitucional\n\n#tuiterosvzla""",0,0,
8776,3er dia #sinluz #volvieron,"3er dia #sinluz #volvieron ""v e r o vampamel"" ""vvampamel"" ""martillera de bossio vampamel propiedades, madre de 2 varones, esposa de un buen compagnero de ruta e hija de 2 personas maravillosas. quê mas puedo pedir?""",0,0,
...,...,...,...,...,...
10996,actualizado: usuarios sin suministro electrico: @oficialedesur @edenorclientes #sinluz #secorto https://t.co/3v7kxspe2r,"actualizado: usuarios sin suministro electrico: @oficialedesur @edenorclientes #sinluz #secorto https://t.co/3v7kxspe2r ""se corto la luz!"" ""se_corto"" ""datos del suministro de luz actualizados cada media hora para capital federal y gran bs as""",0,0,
10997,#ahora las 5 zonas mas afectadas sin suministro electrico por empresa. #sinluz #secorto #luisguillon #virreydelpino https://t.co/b8kg4dgmhs,"#ahora las 5 zonas mas afectadas sin suministro electrico por empresa. #sinluz #secorto #luisguillon #virreydelpino https://t.co/b8kg4dgmhs ""se corto la luz!"" ""se_corto"" ""datos del suministro de luz actualizados cada media hora para capital federal y gran bs as""",0,0,
10998,terceros comodos...\n#sinluz #apagon https://t.co/pentiap1z5,"terceros comodos...\n#sinluz #apagon https://t.co/pentiap1z5 ""claudio"" ""claudio_sotto"" ""se compra lo que tiene precio, lo que tiene valor se conquista. #independiente #reydecopas""",0,0,
10999,"varias zonas de #vargas arriban a mas de 12 horas sin servicio electrico. \n\ndesde ayer en la tarde cuando se genero el incendio en la planta de #tacoa comenzaron las fallas en el suministro.\n\nmaiquetia, naiguata, caraballeda y caribe son algunos sectores que continuan #sinluz","varias zonas de #vargas arriban a mas de 12 horas sin servicio electrico. \n\ndesde ayer en la tarde cuando se genero el incendio en la planta de #tacoa comenzaron las fallas en el suministro.\n\nmaiquetia, naiguata, caraballeda y caribe son algunos sectores que continuan #sinluz ""dsd vargas"" ""dsdvargas"" ""democracia-sociedad y desarrollos para venezuela| capitulo #vargas | #redesdsd | #activismo | somos #ciudadanosdecambio""",0,0,


The list of capital cities in Venezuela is able to extract 2229 rows that mention them.

In [0]:
estados = {"""amazonas|
            anzoategui|
            apure|
            aragua|
            barinas|
            bolivar|
            carabobo|
            cojedes|
            delta Amacuro|
            distrito Capital|
            falcon|
            guarico|
            lara|
            merida|
            miranda|
            monagas|
            nueva Esparta|
            portuguesa|
            sucre|
            tachira|
            trujillo|
            vargas|
            yaracuy|
            zulia"""}

In [39]:
# indices_w_states = to_tag_df.concat_text_user_description.str.contains(estados, case = False)

# to_tag_df[indices_w_states] # 1 row

,full_text,concat_text_user_description,label_country,label_state,label_type
10888,#26dic #amazonas #singasolina no hay gasolina en pto. ayacucho.\nmotorizados trancan la redoma autana en protesta por la falta de combustible https://t.co/mz70qov1un - @carolinaazavach,"#26dic #amazonas #singasolina no hay gasolina en pto. ayacucho.\nmotorizados trancan la redoma autana en protesta por la falta de combustible https://t.co/mz70qov1un - @carolinaazavach ""reporte ya"" ""reporteya"" ""red venezolana de periodismo ciudadano. dictamos talleres para el uso de redes sociales. pioneros en alfabetizacion digital.""",0,0,


In [49]:
if to_tag_df.concat_text_user_description.str.contains(ciudades, case = False) == True:
  to_tag_df['label_state'].replace(ciudades, inplace = True)

else:
   

SyntaxError: ignored